<a href="https://colab.research.google.com/github/vdrakopoulou/vdrakopoulou/blob/main/ESG_messages_from_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

# SETTINGS
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# Display the first 5 rows of the DataFrame
display(df_urls.head())

In [ ]:
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup

# ---------------------------------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------------------------------

def url_points_to_pdf(url: str) -> bool:
    """
    Decide if a URL is (likely) a PDF.
    First check the file extension; if unclear, try a HEAD request.
    """
    try:
        path = urlparse(url).path
        if path.lower().endswith(".pdf"):
            return True

        # Fallback: check Content-Type
        resp = requests.head(url, headers=HEADERS, allow_redirects=True, timeout=15)
        ctype = resp.headers.get("Content-Type", "").lower()
        if "pdf" in ctype:
            return True
    except Exception:
        # On any error, assume it's not a PDF and let HTML logic handle it
        pass
    return False


def get_html(url: str, timeout: int = 15):
    """Download HTML and return BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"  [WARN] Could not fetch HTML {url}: {e}")
        return None

In [ ]:
from urllib.parse import urljoin

def find_pdf_links_on_html_page(page_url: str):
    """
    Find candidate ESG / annual / sustainability report PDFs
    on an HTML ESG page.

    Returns:
        list of absolute PDF URLs (may be empty).
    """
    soup = get_html(page_url)
    if soup is None:
        return []

    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" not in href.lower():
            continue

        full_url = urljoin(page_url, href)
        text = (a.get_text() or "").lower()
        url_low = full_url.lower()

        # Keep links that *look* like reports
        if any(kw in text or kw in url_low
               for kw in ["esg", "sustainab", "csr", "integrated", "annual", "report"]):
            pdf_links.append(full_url)

    # Also look for <iframe> or <embed> with PDFs
    for tag in soup.find_all(["iframe", "embed"], src=True):
        src = tag["src"]
        if ".pdf" in src.lower():
            full_url = urljoin(page_url, src)
            pdf_links.append(full_url)

    return sorted(set(pdf_links))

In [ ]:
import time

def download_pdf(url: str, out_path: str):
    """
    Download a PDF to 'out_path' if it doesn't already exist.

    Returns:
        out_path on success, None on failure.
    """
    if os.path.exists(out_path):
        return out_path

    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        time.sleep(1)  # be polite to servers
        return out_path
    except Exception as e:
        print(f"  [WARN] Failed to download {url}: {e}")
        return None

In [ ]:
!pip install pdfplumber

In [ ]:
import re
import pdfplumber

def simple_sentence_split(text: str):
    """Very simple sentence splitter using punctuation."""
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    parts = re.split(r"(?<=[\.\?\!])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def sentence_has_esg_keyword(sentence: str, keywords=None) -> bool:
    """Return True if the sentence contains at least one ESG keyword."""
    if keywords is None:
        keywords = ESG_KEYWORDS
    s = sentence.lower()
    return any(kw in s for kw in keywords)


def extract_esg_sentences_from_pdf(pdf_path: str):
    """
    Extract ESG-related sentences from a PDF.

    Returns:
        list of dicts: {"page": int, "sentence": str}
    """
    results = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    text = page.extract_text() or ""
                except Exception:
                    text = ""
                if not text.strip():
                    continue

                sentences = simple_sentence_split(text)
                for sent in sentences:
                    if sentence_has_esg_keyword(sent):
                        results.append({"page": page_num, "sentence": sent})
    except Exception as e:
        print(f"  [WARN] Failed to parse PDF {pdf_path}: {e}")
    return results

In [ ]:
def guess_year_from_url(url: str) -> str:
    """Best-effort guess of report year from the URL."""
    m = re.search(r"(20\d{2})", url)
    return m.group(1) if m else ""


def safe_filename(name: str) -> str:
    """Replace characters that are problematic in filenames."""
    return re.sub(r"[^A-Za-z0-9_\-.]", "_", name)

# Task
Save all extracted ESG sentences and their metadata into a CSV file named `esg_messages_from_reports.csv`.

## Save All ESG Sentences to CSV

### Subtask:
Convert the `all_rows` list, which contains all extracted ESG sentences and their metadata, into a pandas DataFrame. Then, save this DataFrame to the specified `OUTPUT_CSV` file, ensuring all collected ESG sentences are persistently stored and accessible.


**Reasoning**:
Convert the `all_rows` list into a pandas DataFrame and then save it to a CSV file as specified in the subtask instructions.



In [ ]:
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")

NameError: name 'pd' is not defined

**Reasoning**:
The previous cell failed because 'pd' was not defined. This indicates that the pandas library, aliased as 'pd', was not imported before being used. I need to explicitly import pandas within this cell to resolve the `NameError`.



In [ ]:
import pandas as pd

df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")

NameError: name 'all_rows' is not defined

**Reasoning**:
The previous code failed because the variable 'all_rows' was not defined. The task requires collecting ESG sentences and their metadata into this list before converting it to a DataFrame. I need to add a step that iterates through the provided URLs, identifies PDF reports, downloads them, extracts ESG sentences, and populates the 'all_rows' list with this data, including company, year, source URL, page, and the sentence itself.



In [ ]:
all_rows = []

for i, row in df_urls.iterrows():
    company = row["company"]
    base_url = row["url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")

NameError: name 'df_urls' is not defined

**Reasoning**:
The previous code failed because 'df_urls' was not defined, leading to a `NameError`. This indicates that the DataFrame was not loaded into memory or its definition was lost. I need to re-include the necessary code to import pandas and os, define the required settings (URL_CSV, OUTPUT_CSV, PDF_DIR), create the directory for PDFs, and load 'df_urls' from 'esg_urls.csv' at the beginning of the code block. This will ensure all dependencies for the main loop and subsequent DataFrame operations are met.



In [ ]:
import pandas as pd
import os

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

all_rows = []

for i, row in df_urls.iterrows():
    company = row["company"]
    base_url = row["url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")

KeyError: 'company'

**Reasoning**:
The previous code failed with a `KeyError: 'company'` because the column name in `df_urls` is `company_name`, not `company`. Additionally, the URL column is `esg_page_url`, not `url`. I need to correct these column names in the loop to match the DataFrame's actual structure.



In [ ]:
import pandas as pd
import os

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

all_rows = []

for i, row in df_urls.iterrows():
    company = row["company_name"]
    base_url = row["esg_page_url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")


Processing Al Buhaira National Insurance Company: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363


NameError: name 'url_points_to_pdf' is not defined

**Reasoning**:
The previous code failed with a `NameError: name 'url_points_to_pdf' is not defined`. This indicates that the helper functions and their necessary imports from previous code cells were not loaded into the current execution environment. To resolve this, I need to consolidate all required imports and function definitions (`url_points_to_pdf`, `get_html`, `find_pdf_links_on_html_page`, `download_pdf`, `simple_sentence_split`, `sentence_has_esg_keyword`, `extract_esg_sentences_from_pdf`, `guess_year_from_url`, `safe_filename`) into this single code block to make them accessible before they are called.



In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse, urljoin
import requests
from bs4 import BeautifulSoup
import time
import re
import pdfplumber

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

# ---------------------------------------------------------------------
# HELPER FUNCTIONS (Copied from previous cells to ensure availability)
# ---------------------------------------------------------------------

def url_points_to_pdf(url: str) -> bool:
    """
    Decide if a URL is (likely) a PDF.
    First check the file extension; if unclear, try a HEAD request.
    """
    try:
        path = urlparse(url).path
        if path.lower().endswith(".pdf"):
            return True

        # Fallback: check Content-Type
        resp = requests.head(url, headers=HEADERS, allow_redirects=True, timeout=15)
        ctype = resp.headers.get("Content-Type", "").lower()
        if "pdf" in ctype:
            return True
    except Exception:
        # On any error, assume it's not a PDF and let HTML logic handle it
        pass
    return False


def get_html(url: str, timeout: int = 15):
    """Download HTML and return BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"  [WARN] Could not fetch HTML {url}: {e}")
        return None


def find_pdf_links_on_html_page(page_url: str):
    """
    Find candidate ESG / annual / sustainability report PDFs
    on an HTML ESG page.

    Returns:
        list of absolute PDF URLs (may be empty).
    """
    soup = get_html(page_url)
    if soup is None:
        return []

    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" not in href.lower():
            continue

        full_url = urljoin(page_url, href)
        text = (a.get_text() or "").lower()
        url_low = full_url.lower()

        # Keep links that *look* like reports
        if any(kw in text or kw in url_low
               for kw in ["esg", "sustainab", "csr", "integrated", "annual", "report"]):
            pdf_links.append(full_url)

    # Also look for <iframe> or <embed> with PDFs
    for tag in soup.find_all(["iframe", "embed"], src=True):
        src = tag["src"]
        if ".pdf" in src.lower():
            full_url = urljoin(page_url, src)
            pdf_links.append(full_url)

    return sorted(set(pdf_links))


def download_pdf(url: str, out_path: str):
    """
    Download a PDF to 'out_path' if it doesn't already exist.

    Returns:
        out_path on success, None on failure.
    """
    if os.path.exists(out_path):
        return out_path

    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        time.sleep(1)  # be polite to servers
        return out_path
    except Exception as e:
        print(f"  [WARN] Failed to download {url}: {e}")
        return None


def simple_sentence_split(text: str):
    """Very simple sentence splitter using punctuation."""
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    parts = re.split(r"(?<=[.?!])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def sentence_has_esg_keyword(sentence: str, keywords=None) -> bool:
    """Return True if the sentence contains at least one ESG keyword."""
    if keywords is None:
        keywords = ESG_KEYWORDS
    s = sentence.lower()
    return any(kw in s for kw in keywords)


def extract_esg_sentences_from_pdf(pdf_path: str):
    """
    Extract ESG-related sentences from a PDF.

    Returns:
        list of dicts: {"page": int, "sentence": str}
    """
    results = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    text = page.extract_text() or ""
                except Exception:
                    text = ""
                if not text.strip():
                    continue

                sentences = simple_sentence_split(text)
                for sent in sentences:
                    if sentence_has_esg_keyword(sent):
                        results.append({"page": page_num, "sentence": sent})
    except Exception as e:
        print(f"  [WARN] Failed to parse PDF {pdf_path}: {e}")
    return results


def guess_year_from_url(url: str) -> str:
    """Best-effort guess of report year from the URL."""
    m = re.search(r"(20\d{2})", url)
    return m.group(1) if m else ""


def safe_filename(name: str) -> str:
    """Replace characters that are problematic in filenames."""
    return re.sub(r"[^A-Za-z0-9_\-.]", "_", name)


# Main processing loop
all_rows = []

for i, row in df_urls.iterrows():
    company = row["company_name"]
    base_url = row["esg_page_url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")

ModuleNotFoundError: No module named 'pdfplumber'

**Reasoning**:
The previous code failed with a `ModuleNotFoundError: No module named 'pdfplumber'`, indicating that the library is not installed. I need to install `pdfplumber` using `!pip install` in a separate cell before any code that imports or uses it.



In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.5 MB/s eta 0:00:00


**Reasoning**:
The `pdfplumber` library has been successfully installed. Now I need to re-run the complete code block that includes all necessary imports, helper functions, and the main processing loop to collect the ESG sentences. This will ensure that `all_rows` is populated with data and that the final DataFrame creation and CSV saving steps can execute without any `NameError` or `ModuleNotFoundError`.



In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse, urljoin
import requests
from bs4 import BeautifulSoup
import time
import re
import pdfplumber

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

# ---------------------------------------------------------------------
# HELPER FUNCTIONS (Copied from previous cells to ensure availability)
# ---------------------------------------------------------------------

def url_points_to_pdf(url: str) -> bool:
    """
    Decide if a URL is (likely) a PDF.
    First check the file extension; if unclear, try a HEAD request.
    """
    try:
        path = urlparse(url).path
        if path.lower().endswith(".pdf"):
            return True

        # Fallback: check Content-Type
        resp = requests.head(url, headers=HEADERS, allow_redirects=True, timeout=15)
        ctype = resp.headers.get("Content-Type", "").lower()
        if "pdf" in ctype:
            return True
    except Exception:
        # On any error, assume it's not a PDF and let HTML logic handle it
        pass
    return False


def get_html(url: str, timeout: int = 15):
    """Download HTML and return BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"  [WARN] Could not fetch HTML {url}: {e}")
        return None


def find_pdf_links_on_html_page(page_url: str):
    """
    Find candidate ESG / annual / sustainability report PDFs
    on an HTML ESG page.

    Returns:
        list of absolute PDF URLs (may be empty).
    """
    soup = get_html(page_url)
    if soup is None:
        return []

    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" not in href.lower():
            continue

        full_url = urljoin(page_url, href)
        text = (a.get_text() or "").lower()
        url_low = full_url.lower()

        # Keep links that *look* like reports
        if any(kw in text or kw in url_low
               for kw in ["esg", "sustainab", "csr", "integrated", "annual", "report"]):
            pdf_links.append(full_url)

    # Also look for <iframe> or <embed> with PDFs
    for tag in soup.find_all(["iframe", "embed"], src=True):
        src = tag["src"]
        if ".pdf" in src.lower():
            full_url = urljoin(page_url, src)
            pdf_links.append(full_url)

    return sorted(set(pdf_links))


def download_pdf(url: str, out_path: str):
    """
    Download a PDF to 'out_path' if it doesn't already exist.

    Returns:
        out_path on success, None on failure.
    """
    if os.path.exists(out_path):
        return out_path

    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        time.sleep(1)  # be polite to servers
        return out_path
    except Exception as e:
        print(f"  [WARN] Failed to download {url}: {e}")
        return None


def simple_sentence_split(text: str):
    """Very simple sentence splitter using punctuation."""
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    parts = re.split(r"(?<=[.?!])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def sentence_has_esg_keyword(sentence: str, keywords=None) -> bool:
    """Return True if the sentence contains at least one ESG keyword."""
    if keywords is None:
        keywords = ESG_KEYWORDS
    s = sentence.lower()
    return any(kw in s for kw in keywords)


def extract_esg_sentences_from_pdf(pdf_path: str):
    """
    Extract ESG-related sentences from a PDF.

    Returns:
        list of dicts: {"page": int, "sentence": str}
    """
    results = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    text = page.extract_text() or ""
                except Exception:
                    text = ""
                if not text.strip():
                    continue

                sentences = simple_sentence_split(text)
                for sent in sentences:
                    if sentence_has_esg_keyword(sent):
                        results.append({"page": page_num, "sentence": sent})
    except Exception as e:
        print(f"  [WARN] Failed to parse PDF {pdf_path}: {e}")
    return results


def guess_year_from_url(url: str) -> str:
    """Best-effort guess of report year from the URL."""
    m = re.search(r"(20\d{2})", url)
    return m.group(1) if m else ""


def safe_filename(name: str) -> str:
    """Replace characters that are problematic in filenames."""
    return re.sub(r"[^A-Za-z0-9_\-.]", "_", name)


# Main processing loop
all_rows = []

for i, row in df_urls.iterrows():
    company = row["company_name"]
    base_url = row["esg_page_url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")


Processing Al Buhaira National Insurance Company: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363: 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363

Processing Abu Dhabi Aviation Co.: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  [INFO] Base URL is a PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  Downloading/Processing PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_Aviation_Co._2025_Sustainability-Report-2024-English.pdf

Processing Abu Dhabi Commercial Bank

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3074447_ØªÙ\x82Ø±Ù\x8aØ± Ø§Ù\x84Ø¥Ø³ØªØ¯Ø§Ù\x85Ù\x87 EN.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="-1048613850"\r\nSet-Cookie: dtCookie=v_4_srv_3_sn_5558C1B714E2A24A2C923E99838C082D_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:08:21 GMT\r\nSet-Cookie: adxservices-wafpersistence=!aCGAALk+MHSsX0yLfuIn2Oot7nTWAjoFWQEh9toRWqn1DGDQ5kKdOg0IyIutayGhxZk5hK8BO/cCqw==; path=/; Httponly; Secure\r\nSet-Cookie: TS010

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3074447_ØªÙ\x82Ø±Ù\x8aØ± Ø§Ù\x84Ø¥Ø³ØªØ¯Ø§Ù\x85Ù\x87 EN.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="-526016214"\r\nSet-Cookie: dtCookie=v_4_srv_3_sn_2E2CC79FCD6CBFB0FA0139B97B681C63_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:08:23 GMT\r\nSet-Cookie: adxservices-wafpersistence=!Exr2eNorOB9PA3uLfuIn2Oot7nTWAiFVLUUP0dwqP4kz1DVuqGIA7HBOC/9NpxvtYG+HJ5ulGJ0wwQ==; path=/; Httponly; Secure\r\nSet-Cookie: TS0107

    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_National_Co._for_Building_Materials__contentdownload.aspx_doc_3074447



Processing Borouge PLC: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [INFO] Searching for PDFs on HTML page: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [WARN] No relevant PDF links found on https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 

Processing Bank of Sharjah: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [INFO] Searching for PDFs on HTML page: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [WARN] Could not fetch HTML https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf : 403 Client Error: Forbidden for url: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Re

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=EasyLease_ESG_Report_2023.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="66328363"\r\nSet-Cookie: dtCookie=v_4_srv_1_sn_7AAD081CD8AE757BB33E820A2D15158E_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:09:50 GMT\r\nSet-Cookie: adxservices-wafpersistence=!avndTLBNgK6mR4SwdSX6FNppFSHBAmJlo31KCcz4suWcvzxS2zrdEphIgLFr6dPcZl0ajgnNCKjsLg==; path=/; Httponly; Secure\r\nSet-Cookie: TS01075b0f=019c04258c97112f336651d4481

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=EasyLease_ESG_Report_2023.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="-2076040091"\r\nSet-Cookie: dtCookie=v_4_srv_4_sn_209BE8EEE9E095DC74594B56AF56ED59_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:09:59 GMT\r\nSet-Cookie: adxservices-wafpersistence=!H6RXvhzWTJH4jw+wdSX6FNppFSHBAodukI0guA8DSuk37XgKxxXLFk85eLHj/qM4woiHprxy88Rl3g==; path=/; Httponly; Secure\r\nSet-Cookie: TS01075b0f=019c04258c9abff4985701db

  [WARN] Failed to download https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 : HTTPSConnectionPool(host='adxservices.adx.ae', port=443): Read timed out.
    Skipping sentence extraction for https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216  due to download failure.

Processing Emirates Insurance Co.: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 : 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856%20
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 

Processing Emsteel Building Materials PJSC: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4294570 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/

**Reasoning**:
The `pdfplumber` library has been successfully installed. Now I need to re-run the complete code block that includes all necessary imports, helper functions, and the main processing loop to collect the ESG sentences. This will ensure that `all_rows` is populated with data and that the final DataFrame creation and CSV saving steps can execute without any `NameError` or `ModuleNotFoundError`.



In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse, urljoin
import requests
from bs4 import BeautifulSoup
import time
import re
import pdfplumber

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

# ---------------------------------------------------------------------
# HELPER FUNCTIONS (Copied from previous cells to ensure availability)
# ---------------------------------------------------------------------

def url_points_to_pdf(url: str) -> bool:
    """
    Decide if a URL is (likely) a PDF.
    First check the file extension; if unclear, try a HEAD request.
    """
    try:
        path = urlparse(url).path
        if path.lower().endswith(".pdf"):
            return True

        # Fallback: check Content-Type
        resp = requests.head(url, headers=HEADERS, allow_redirects=True, timeout=15)
        ctype = resp.headers.get("Content-Type", "").lower()
        if "pdf" in ctype:
            return True
    except Exception:
        # On any error, assume it's not a PDF and let HTML logic handle it
        pass
    return False


def get_html(url: str, timeout: int = 15):
    """Download HTML and return BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"  [WARN] Could not fetch HTML {url}: {e}")
        return None


def find_pdf_links_on_html_page(page_url: str):
    """
    Find candidate ESG / annual / sustainability report PDFs
    on an HTML ESG page.

    Returns:
        list of absolute PDF URLs (may be empty).
    """
    soup = get_html(page_url)
    if soup is None:
        return []

    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" not in href.lower():
            continue

        full_url = urljoin(page_url, href)
        text = (a.get_text() or "").lower()
        url_low = full_url.lower()

        # Keep links that *look* like reports
        if any(kw in text or kw in url_low
               for kw in ["esg", "sustainab", "csr", "integrated", "annual", "report"]):
            pdf_links.append(full_url)

    # Also look for <iframe> or <embed> with PDFs
    for tag in soup.find_all(["iframe", "embed"], src=True):
        src = tag["src"]
        if ".pdf" in src.lower():
            full_url = urljoin(page_url, src)
            pdf_links.append(full_url)

    return sorted(set(pdf_links))


def download_pdf(url: str, out_path: str):
    """
    Download a PDF to 'out_path' if it doesn't already exist.

    Returns:
        out_path on success, None on failure.
    """
    if os.path.exists(out_path):
        return out_path

    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        time.sleep(1)  # be polite to servers
        return out_path
    except Exception as e:
        print(f"  [WARN] Failed to download {url}: {e}")
        return None


def simple_sentence_split(text: str):
    """Very simple sentence splitter using punctuation."""
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    parts = re.split(r"(?<=[.?!])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def sentence_has_esg_keyword(sentence: str, keywords=None) -> bool:
    """Return True if the sentence contains at least one ESG keyword."""
    if keywords is None:
        keywords = ESG_KEYWORDS
    s = sentence.lower()
    return any(kw in s for kw in keywords)


def extract_esg_sentences_from_pdf(pdf_path: str):
    """
    Extract ESG-related sentences from a PDF.

    Returns:
        list of dicts: {"page": int, "sentence": str}
    """
    results = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    text = page.extract_text() or ""
                except Exception:
                    text = ""
                if not text.strip():
                    continue

                sentences = simple_sentence_split(text)
                for sent in sentences:
                    if sentence_has_esg_keyword(sent):
                        results.append({"page": page_num, "sentence": sent})
    except Exception as e:
        print(f"  [WARN] Failed to parse PDF {pdf_path}: {e}")
    return results


def guess_year_from_url(url: str) -> str:
    """Best-effort guess of report year from the URL."""
    m = re.search(r"(20\d{2})", url)
    return m.group(1) if m else ""


def safe_filename(name: str) -> str:
    """Replace characters that are problematic in filenames."""
    return re.sub(r"[^A-Za-z0-9_\-.]", "_", name)


# Main processing loop
all_rows = []

for i, row in df_urls.iterrows():
    company = row["company_name"]
    base_url = row["esg_page_url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")


Processing Al Buhaira National Insurance Company: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363: 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363

Processing Abu Dhabi Aviation Co.: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  [INFO] Base URL is a PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  Downloading/Processing PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_Aviation_Co._2025_Sustainability-Report-2024-English.pdf

Processing Abu Dhabi Commercial Bank

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3074447_ØªÙ\x82Ø±Ù\x8aØ± Ø§Ù\x84Ø¥Ø³ØªØ¯Ø§Ù\x85Ù\x87 EN.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="68609540"\r\nSet-Cookie: dtCookie=v_4_srv_4_sn_E7A8C1540EEA6BE91BCD8FA8EA827018_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:27:06 GMT\r\nSet-Cookie: adxservices-wafpersistence=!xy79IYkDFqX4W1GwdSX6FNppFSHBArs30zZY8F0Y9VQSQ8meYjaxOaIOJFh3Tsf68Khz7sZZun61tg==; path=/; Httponly; Secure\r\nSet-Cookie: TS01075b

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447
    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_National_Co._for_Building_Materials__contentdownload.aspx_doc_3074447



Processing Borouge PLC: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [INFO] Searching for PDFs on HTML page: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [WARN] No relevant PDF links found on https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 

Processing Bank of Sharjah: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [INFO] Searching for PDFs on HTML page: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [WARN] Could not fetch HTML https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf : 403 Client Error: Forbidden for url: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Re

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3182216 _EasyLease_ESG_Report_2023.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="1413479570"\r\nSet-Cookie: dtCookie=v_4_srv_1_sn_D2D4242216CFDCB98AB8A718D59A8CBF_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:28:06 GMT\r\nSet-Cookie: adxservices-wafpersistence=!sOvo8qKiKpHaZLiwdSX6FNppFSHBAlCauz31GTALX3zXqOAa4Smn7OBkFyRtZwV5fEy4sRIqCIRgbg==; path=/; Httponly; Secure\r\nSet-Cookie: TS01075b0f=019c04258c087c8a

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 
    Extracting ESG sentences from esg_pdfs/Easy_Lease_Motorcycle_Rental_PJSC__contentdownload.aspx_doc_3182216_

Processing Emirates Insurance Co.: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 : 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856%20
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 

Processing Emsteel Building Materials PJSC: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4294570 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/con

**Reasoning**:
The `pdfplumber` library has been successfully installed. Now I need to re-run the complete code block that includes all necessary imports, helper functions, and the main processing loop to collect the ESG sentences. This will ensure that `all_rows` is populated with data and that the final DataFrame creation and CSV saving steps can execute without any `NameError` or `ModuleNotFoundError`.



In [ ]:
import pandas as pd
import os
from urllib.parse import urlparse, urljoin
import requests
from bs4 import BeautifulSoup
import time
import re
import pdfplumber

# SETTINGS (necessary for df_urls and output)
URL_CSV = "esg_urls.csv"
OUTPUT_CSV = "esg_messages_from_reports.csv"
PDF_DIR = "esg_pdfs"

os.makedirs(PDF_DIR, exist_ok=True)

# Load the CSV file into a pandas DataFrame
df_urls = pd.read_csv(URL_CSV)

# ESG_KEYWORDS and HEADERS are also used by helper functions in this block
ESG_KEYWORDS = [
    "esg", "sustainab", "environment", "emission", "climate",
    "carbon", "co2", "net zero", "net-zero", "green", "csr",
    "social", "governance", "diversity", "inclusion", "water",
    "waste", "renewable", "energy", "human rights"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ESG-Collector/1.0)"
}

# ---------------------------------------------------------------------
# HELPER FUNCTIONS (Copied from previous cells to ensure availability)
# ---------------------------------------------------------------------

def url_points_to_pdf(url: str) -> bool:
    """
    Decide if a URL is (likely) a PDF.
    First check the file extension; if unclear, try a HEAD request.
    """
    try:
        path = urlparse(url).path
        if path.lower().endswith(".pdf"):
            return True

        # Fallback: check Content-Type
        resp = requests.head(url, headers=HEADERS, allow_redirects=True, timeout=15)
        ctype = resp.headers.get("Content-Type", "").lower()
        if "pdf" in ctype:
            return True
    except Exception:
        # On any error, assume it's not a PDF and let HTML logic handle it
        pass
    return False


def get_html(url: str, timeout: int = 15):
    """Download HTML and return BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"  [WARN] Could not fetch HTML {url}: {e}")
        return None


def find_pdf_links_on_html_page(page_url: str):
    """
    Find candidate ESG / annual / sustainability report PDFs
    on an HTML ESG page.

    Returns:
        list of absolute PDF URLs (may be empty).
    """
    soup = get_html(page_url)
    if soup is None:
        return []

    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ".pdf" not in href.lower():
            continue

        full_url = urljoin(page_url, href)
        text = (a.get_text() or "").lower()
        url_low = full_url.lower()

        # Keep links that *look* like reports
        if any(kw in text or kw in url_low
               for kw in ["esg", "sustainab", "csr", "integrated", "annual", "report"]):
            pdf_links.append(full_url)

    # Also look for <iframe> or <embed> with PDFs
    for tag in soup.find_all(["iframe", "embed"], src=True):
        src = tag["src"]
        if ".pdf" in src.lower():
            full_url = urljoin(page_url, src)
            pdf_links.append(full_url)

    return sorted(set(pdf_links))


def download_pdf(url: str, out_path: str):
    """
    Download a PDF to 'out_path' if it doesn't already exist.

    Returns:
        out_path on success, None on failure.
    """
    if os.path.exists(out_path):
        return out_path

    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        time.sleep(1)  # be polite to servers
        return out_path
    except Exception as e:
        print(f"  [WARN] Failed to download {url}: {e}")
        return None


def simple_sentence_split(text: str):
    """Very simple sentence splitter using punctuation."""
    text = re.sub(r"\s+", " ", text)  # collapse whitespace
    parts = re.split(r"(?<=[.?!])\s+", text)
    return [p.strip() for p in parts if p.strip()]


def sentence_has_esg_keyword(sentence: str, keywords=None) -> bool:
    """Return True if the sentence contains at least one ESG keyword."""
    if keywords is None:
        keywords = ESG_KEYWORDS
    s = sentence.lower()
    return any(kw in s for kw in keywords)


def extract_esg_sentences_from_pdf(pdf_path: str):
    """
    Extract ESG-related sentences from a PDF.

    Returns:
        list of dicts: {"page": int, "sentence": str}
    """
    results = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                try:
                    text = page.extract_text() or ""
                except Exception:
                    text = ""
                if not text.strip():
                    continue

                sentences = simple_sentence_split(text)
                for sent in sentences:
                    if sentence_has_esg_keyword(sent):
                        results.append({"page": page_num, "sentence": sent})
    except Exception as e:
        print(f"  [WARN] Failed to parse PDF {pdf_path}: {e}")
    return results


def guess_year_from_url(url: str) -> str:
    """Best-effort guess of report year from the URL."""
    m = re.search(r"(20\d{2})", url)
    return m.group(1) if m else ""


def safe_filename(name: str) -> str:
    """Replace characters that are problematic in filenames."""
    return re.sub(r"[^A-Za-z0-9_\-.]", "_", name)


# Main processing loop
all_rows = []

for i, row in df_urls.iterrows():
    company = row["company_name"]
    base_url = row["esg_page_url"]
    print(f"\nProcessing {company}: {base_url}")

    report_urls = []
    if url_points_to_pdf(base_url):
        report_urls.append(base_url)
        print(f"  [INFO] Base URL is a PDF: {base_url}")
    else:
        # Try to find PDF links on the HTML page
        print(f"  [INFO] Searching for PDFs on HTML page: {base_url}")
        found_links = find_pdf_links_on_html_page(base_url)
        if found_links:
            report_urls.extend(found_links)
        else:
            print(f"  [WARN] No relevant PDF links found on {base_url}")

    if not report_urls:
        continue

    for report_url in report_urls:
        year = guess_year_from_url(report_url)
        filename = safe_filename(f"{company}_{year}_{os.path.basename(report_url)}")
        pdf_path = os.path.join(PDF_DIR, filename)

        print(f"  Downloading/Processing PDF: {report_url}")
        downloaded_path = download_pdf(report_url, pdf_path)

        if downloaded_path:
            print(f"    Extracting ESG sentences from {downloaded_path}")
            extracted_sentences = extract_esg_sentences_from_pdf(downloaded_path)

            for sent_data in extracted_sentences:
                all_rows.append({
                    "company": company,
                    "year": year,
                    "source_url": report_url,
                    "pdf_path": downloaded_path,
                    "page": sent_data["page"],
                    "sentence": sent_data["sentence"],
                })
        else:
            print(f"    Skipping sentence extraction for {report_url} due to download failure.")

print(f"\nFinished processing all URLs. Collected {len(all_rows)} ESG sentences.")
df_esg_sentences = pd.DataFrame(all_rows)
df_esg_sentences.to_csv(OUTPUT_CSV, index=False)
print(f"All ESG sentences saved to {OUTPUT_CSV}")


Processing Al Buhaira National Insurance Company: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363: 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4307363

Processing Abu Dhabi Aviation Co.: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  [INFO] Base URL is a PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
  Downloading/Processing PDF: https://ada.ae/wp-content/uploads/2025/03/Sustainability-Report-2024-English.pdf
    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_Aviation_Co._2025_Sustainability-Report-2024-English.pdf

Processing Abu Dhabi Commercial Bank

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3074447_ØªÙ\x82Ø±Ù\x8aØ± Ø§Ù\x84Ø¥Ø³ØªØ¯Ø§Ù\x85Ù\x87 EN.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="1782048235"\r\nSet-Cookie: dtCookie=v_4_srv_3_sn_6D69B96396AEE0636BC22435F21EAA59_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:37:24 GMT\r\nSet-Cookie: adxservices-wafpersistence=!VXcJZxsUccjtvtewdSX6FNppFSHBAhv1rRbuM0IIS71kuONF/NJUdmsWSLn1LMwcybCDQa7bQokUDw==; path=/; Httponly; Secure\r\nSet-Cookie: TS0107

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3074447
    Extracting ESG sentences from esg_pdfs/Abu_Dhabi_National_Co._for_Building_Materials__contentdownload.aspx_doc_3074447



Processing Borouge PLC: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [INFO] Searching for PDFs on HTML page: https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 
  [WARN] No relevant PDF links found on https://www.borouge.com/en/sustainability/SustainabilityReport/Borouge%20Sustainability%20Report%202024.pdf 

Processing Bank of Sharjah: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [INFO] Searching for PDFs on HTML page: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf 
  [WARN] Could not fetch HTML https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Report-2024-Edition-ENG-Version-200325-ID-4511781.pdf : 403 Client Error: Forbidden for url: https://cdn.bankofsharjah.com/Live/public/uploads/events/BOS-ESG-Re

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 568, in getresponse
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/response.py", line 88, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'Transfer - Encoding: chunked\r\ncontent-disposition: inline; filename=3182216 _EasyLease_ESG_Report_2023.pdf\r\nServer-Timing: dtSInfo;desc="0", dtRpid;desc="-95913079"\r\nSet-Cookie: dtCookie=v_4_srv_4_sn_F027A9F46862FDF2163DABDC111ACE91_perc_100000_ol_0_mul_1_app-3A2dffb00433f451c6_0; Path=/; Domain=.adx.ae\r\nDate: Fri, 06 Feb 2026 15:39:04 GMT\r\nSet-Cookie: adxservices-wafpersistence=!Qs5AvdAGvBh8XmqLfuIn2Oot7nTWAlUYZ5jaDodQixMAqWp19umYK2B9xMBF0D2EYGBZ+ydqWMHGcw==; path=/; Httponly; Secure\r\nSet-Cookie: TS01075b0f=019c04258c08959d9

  [INFO] Base URL is a PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 
  Downloading/Processing PDF: https://adxservices.adx.ae/cdn/contentdownload.aspx?doc=3182216 
    Extracting ESG sentences from esg_pdfs/Easy_Lease_Motorcycle_Rental_PJSC__contentdownload.aspx_doc_3182216_

Processing Emirates Insurance Co.: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 
  [WARN] Could not fetch HTML https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 : 403 Client Error: Forbidden for url: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856%20
  [WARN] No relevant PDF links found on https://apigateway.adx.ae/adx/cdn/1.0/content/download/4311856 

Processing Emsteel Building Materials PJSC: https://apigateway.adx.ae/adx/cdn/1.0/content/download/4294570 
  [INFO] Searching for PDFs on HTML page: https://apigateway.adx.ae/adx/cdn/1.0/con